In [17]:
"""Detector de ZDIRECTION multilínea (robusto y en streaming).

Reglas:
- Un registro NUEVO comienza si, tras espacios iniciales opcionales,
  la línea cumple r'^[A-Z0-9_]+;' (ej.: 'E1BPEXTC;...', 'E1SHP_OBDLV_SAVE_REPLICA;...').
- Si una línea NO cumple ese patrón, es 'continuación' del registro previo.
- Se reportan registros que contienen 'ZDIRECTION;' y abarcan > 1 línea física.
"""

import re
from typing import List, Dict, Optional

# Patrón más permisivo: admite espacios iniciales y guion bajo en el prefijo
RE_RECORD_START = re.compile(r"^\s*[A-Z0-9_]+;")
TOKEN = "ZDIRECTION;"  # buscar con ';' para evitar falsos positivos
ARCHIVO = "SHP_OBDLV_SAVE_REPLICA_20250916094607552001.TXT"


def find_multiline_zdirection_streaming(
    filepath: str,
    show_progress_every: Optional[int] = None,
) -> List[Dict[str, object]]:
    """
    Procesa el archivo en una sola pasada, sin cargarlo completo a memoria.

    Parámetros
    ----------
    filepath : str
        Ruta del archivo a analizar.
    show_progress_every : int | None
        Si se entrega N, imprime progreso cada N líneas.

    Retorna
    -------
    list[dict]
        Registros con:
            - start_line, end_line, num_lines, record_text
    """
    results: List[Dict[str, object]] = []

    # Estado del registro actual
    cur_start_line: Optional[int] = None
    cur_lines: List[str] = []
    cur_has_token = False

    def flush_current(end_line: int):
        """Cierra el registro actual y lo agrega a resultados si aplica."""
        nonlocal cur_start_line, cur_lines, cur_has_token

        if cur_start_line is not None:
            num = len(cur_lines)
            if cur_has_token and num > 1:
                results.append(
                    {
                        "start_line": cur_start_line,
                        "end_line": end_line,
                        "num_lines": num,
                        "record_text": "\n".join(cur_lines),
                    }
                )
        # Reset del estado
        cur_start_line = None
        cur_lines = []
        cur_has_token = False

    with open(filepath, "r", encoding="utf-8", errors="replace", newline="") as f:
        for line_no, raw in enumerate(f, start=1):
            # Conserva la línea “visual” pero sin fin de línea
            line = raw.rstrip("\r\n")

            # Progreso opcional
            if show_progress_every and (line_no % show_progress_every == 0):
                print(f"[info] Procesadas {line_no:,} líneas…")

            if RE_RECORD_START.match(line):
                # Nuevo inicio: cerramos (si hay) el registro anterior
                if cur_start_line is not None:
                    flush_current(end_line=line_no - 1)

                # Abrimos nuevo registro
                cur_start_line = line_no
                cur_lines = [line]
                cur_has_token = TOKEN in line
            else:
                # Continuación del registro actual (si existe)
                if cur_start_line is None:
                    # Línea suelta sin cabecera previa: ignorar
                    continue
                cur_lines.append(line)
                if not cur_has_token and TOKEN in line:
                    cur_has_token = True

        # Fin de archivo: cerrar el último registro si quedó abierto
        if cur_start_line is not None:
            flush_current(end_line=line_no)

    return results


# ===== Ejecución =====
if __name__ == "__main__":
    hallazgos = find_multiline_zdirection_streaming(
        ARCHIVO,
        show_progress_every=None,  # pon un entero para ver progreso
    )

    if not hallazgos:
        print("No se encontraron registros ZDIRECTION multilínea.")
    else:
        print(f"Se encontraron {len(hallazgos)} registro(s) ZDIRECTION multilínea:\n")
        for rec in hallazgos:
            print(
                f"- Desde línea {rec['start_line']} hasta {rec['end_line']} "
                f"({rec['num_lines']} líneas):"
            )
            print(rec["record_text"])
            print("-" * 80)



No se encontraron registros ZDIRECTION multilínea.
